## Imports

In [2]:
import pandas as pd
import numpy as np

import nltk

import re

import spacy

import matplotlib.pyplot as plt

#### Reading the File

In [3]:
df = pd.read_excel('Documents.xlsx')

#### managing twits lengths

In [4]:
df['length'] = [len(x) for x in df['Title']]
max_length = int(np.max(df['length']))

In [5]:
max_length

536

## Preprocessing

#### Managing hyperlinks

In [6]:
def get_hyperlinks(text):
    ## finds all hyperlinks in the text
    return re.findall(r"http\S+", text)

def rmv_hyperlink(text):
    ## removes all hyperlinks of the text
    return re.sub(r"http\S+", "", text)


df['hyperlinks'] = df['Title'].map(
    get_hyperlinks
)

df['Title'] = df['Title'].map(
    rmv_hyperlink
)

#### Managing hashtags

In [7]:
def get_hashtags(text):
    ## finds all hashtags of the text
    return re.findall(r"#\S+", text)

def rmv_hashtags(text):
    ## removes all hashtags of the text
    return re.sub(r"#\S+", "", text)

In [8]:
df['hashtags'] = df['Title'].map(
    get_hashtags
)

df['Title'] = df['Title'].map(
    rmv_hashtags
)

#### Managing user references '@'

In [9]:
def get_users(text):
    return re.findall(r"@\S+", text)

In [10]:
df['referenced_users'] = df['Title'].map(
    get_users
)

#### tokenizing in sentences and PoS

In [11]:
## french PoS:
## https://github.com/cmchurch/nltk_french/blob/master/french-nltk.py

## https://nlp.stanford.edu/software/CRF-NER.html

nlp = spacy.load('fr_core_news_sm')

df['Title'].map(
    nltk.sent_tokenize
)

0              [Il a guéri du COVID ou il a tué Thanos ?]
1       [les premiers symptômes du coronavirus c’est l...
2       [Je souhaite une agréable journée à tout le mo...
3                 [Le COVID va bientôt fêter ses 1ans la]
4       [Le covid il a passé plus de temps que moi à l...
                              ...                        
9995    [[ Le taux de positivité chez les patients sym...
9996    [La @VilledeGrenoble, via le CCAS, a distribué...
9997    [Considérée comme un "cas contact", Roselyne B...
9998    [Bonjour, test covid très douloureux ce matin....
9999    [HAHA\n\nUne main apparaît dans le champ de vi...
Name: Title, Length: 10000, dtype: object

#### Example of spacy

In [12]:
sample_text = """Mais ces derniers sont loin d’exclure l’idée que le philosophe s’intéresse à des problèmes théoriques. La « sagesse », ou plus exactement la sophia, que souhaite appréhender le philosophe est aussi un savoir et une connaissance. Le philosophe, dans la lignée de la tradition fondée par Socrate, sait comment il doit vivre ; il peut justifier ses choix et son mode de vie. Socrate par exemple, dans les dialogues socratiques de Platon, exige de ses interlocuteurs qu’ils soient à même de donner le logos de leur jugement de valeur et de leur choix, c’est-à-dire de les justifier rationnellement. Cette exigence de rationalité peut même amener à donner des fondements authentiquement scientifiques à la philosophie. Bien sûr la définition de la philosophie en tant que modus vivendi (mode de vie) ne peut prétendre être suffisante pour définir la philosophie dans son ensemble. Bien des philosophes ont compris la philosophie comme un travail intellectuel et non comme un mode de vie : c'est le cas dans le monde universitaire et de la recherche de nos jours. Il en va tout autrement, en Inde notamment. Le point de vue occidental ne peut s'appliquer aux concepts philosophiques en vigueur dans cette partie du monde, bien qu'il y eût tentative d'assimilation à l'époque romaine, en particulier avec Plotin."""

docs = [nlp(s) for s in nltk.sent_tokenize(sample_text)]

for (i,doc) in enumerate(docs):
    print('--{}--'.format(i))
    print(doc)
    for entity in doc.ents:
        print('entity: {} || entity label: {}'.format(entity, entity.label_))

--0--
Mais ces derniers sont loin d’exclure l’idée que le philosophe s’intéresse à des problèmes théoriques.
--1--
La « sagesse », ou plus exactement la sophia, que souhaite appréhender le philosophe est aussi un savoir et une connaissance.
entity: » || entity label: MISC
--2--
Le philosophe, dans la lignée de la tradition fondée par Socrate, sait comment il doit vivre ; il peut justifier ses choix et son mode de vie.
entity: Socrate || entity label: PER
--3--
Socrate par exemple, dans les dialogues socratiques de Platon, exige de ses interlocuteurs qu’ils soient à même de donner le logos de leur jugement de valeur et de leur choix, c’est-à-dire de les justifier rationnellement.
entity: Socrate || entity label: PER
entity: Platon || entity label: PER
--4--
Cette exigence de rationalité peut même amener à donner des fondements authentiquement scientifiques à la philosophie.
--5--
Bien sûr la définition de la philosophie en tant que modus vivendi (mode de vie) ne peut prétendre être suff

### Creating vocabulary

#### Lemmatize

In [13]:
from spacy_lefff import LefffLemmatizer, POSTagger

pos = POSTagger()

french_lemmatizer = LefffLemmatizer(after_melt = True)

nlp.add_pipe(pos, name='pos', after='parser')
nlp.add_pipe(french_lemmatizer, name='lefff', after='pos')

2020-10-21 10:14:25,957 - spacy_lefff.downloader - INFO - data already set up
2020-10-21 10:14:25,957 - spacy_lefff.melt_tagger - INFO -   TAGGER: Loading lexicon...
2020-10-21 10:14:26,353 - spacy_lefff.melt_tagger - INFO -   TAGGER: Loading tags...
2020-10-21 10:14:26,381 - spacy_lefff.melt_tagger - INFO -   TAGGER: Loading model from c:\users\artorias_doge\appdata\local\programs\python\python37\lib\site-packages\spacy_lefff\data\tagger\models/fr...
2020-10-21 10:14:27,218 - spacy_lefff.melt_tagger - INFO -   TAGGER: Loading model from c:\users\artorias_doge\appdata\local\programs\python\python37\lib\site-packages\spacy_lefff\data\tagger\models/fr: done
2020-10-21 10:14:27,219 - spacy_lefff.lefff - INFO - New LefffLemmatizer instantiated.
2020-10-21 10:14:27,337 - spacy_lefff.lefff - INFO - Reading lefff data...
2020-10-21 10:14:27,716 - spacy_lefff.lefff - INFO - Successfully loaded lefff lemmatizer


In [14]:
def lemmatize(text):
    return [doc.lemma_ for doc in nlp(text)]
    

lemmatized = df['Title'].map(
    lemmatize
)

2020-10-21 10:14:27,728 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:27,746 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:27,762 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:27,776 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:27,789 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:27,803 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:27,820 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:27,844 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:27,869 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:27,884 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:27,902 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:27,914 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tag

2020-10-21 10:14:29,816 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:29,826 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:29,842 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:29,869 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:29,901 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:29,926 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:29,949 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:29,974 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:29,994 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:30,016 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:30,025 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:30,035 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tag

2020-10-21 10:14:31,956 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:31,984 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:32,008 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:32,031 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:32,050 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:32,076 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:32,097 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:32,122 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:32,147 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:32,179 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:32,220 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:32,239 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tag

2020-10-21 10:14:34,046 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:34,069 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:34,082 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:34,100 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:34,123 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:34,153 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:34,168 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:34,188 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:34,215 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:34,234 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:34,253 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:34,274 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tag

2020-10-21 10:14:36,094 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:36,115 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:36,137 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:36,156 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:36,181 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:36,210 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:36,231 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:36,248 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:36,270 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:36,295 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:36,314 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:14:36,327 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tag

...

2020-10-21 10:17:21,111 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:17:21,125 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:17:21,138 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:17:21,170 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:17:21,192 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:17:21,210 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:17:21,237 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:17:21,257 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:17:21,268 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:17:21,284 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:17:21,304 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:17:21,316 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tag

2020-10-21 10:17:22,720 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:17:22,744 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:17:22,764 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:17:22,789 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:17:22,802 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:17:22,821 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:17:22,836 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:17:22,862 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:17:22,871 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:17:22,887 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:17:22,902 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:17:22,920 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tag

2020-10-21 10:17:24,604 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:17:24,612 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:17:24,620 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:17:24,631 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:17:24,641 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:17:24,661 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:17:24,679 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:17:24,701 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:17:24,709 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:17:24,721 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:17:24,738 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:17:24,751 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tag

2020-10-21 10:17:26,178 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:17:26,189 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:17:26,201 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:17:26,214 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:17:26,227 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:17:26,245 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:17:26,256 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:17:26,266 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:17:26,276 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:17:26,289 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:17:26,303 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:17:26,314 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tag

2020-10-21 10:17:27,917 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:17:27,930 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:17:27,945 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...
2020-10-21 10:17:27,963 - spacy_lefff.melt_tagger - INFO -   TAGGER: POS Tagging...


In [15]:
lemmatized

0       [il, avoir, guérir, de, covid, ou, il, avoir, ...
1       [le, premier, symptôme, de, coronavirus, c’, ê...
2       [je, souhaite, un, agréable, journer, à, tout,...
3       [le, covid, aller, bientôt, fêter, son, 1ans, le]
4       [le, covid, il, avoir, passer, plus, de, temps...
                              ...                        
9995    [[, le, taux, de, positivité, chez, le, patien...
9996    [le, @villedegrenoble, ,, via, le, cca, ,, avo...
9997    [considérer, comme, un, ", cas, contact, ", ,,...
9998    [Bonjour, ,, test, covid, très, douloureux, ce...
9999    [haha, \n\n, un, main, apparaître, dans, le, c...
Name: Title, Length: 10000, dtype: object

In [16]:
lemmatized = lemmatized.map(
    lambda x : " ".join(x)
)

#### Removing unwanted characters

In [17]:
def rmv_char(text):
    ## removes all unwanted characters of the text
    text = re.sub("""[\n´]""", " ", text)
    text = re.sub("""["'+"*%&/()"=¦@#°§¬|¢\[\]\-\_\—‘’“”•`\^~?.,:!$;«»<>]""", "", text)
    return re.sub("""\d""", " ", text)

In [18]:
filtered = lemmatized.map(
    lambda r: r.lower()
).map(
    rmv_char
)
filtered

0       il avoir guérir de covid ou il avoir tuer thanos 
1       le premier symptôme de coronavirus c être le p...
2       je souhaite un agréable journer à tout le mond...
3                le covid aller bientôt fêter son  ans le
4       le covid il avoir passer plus de temps que moi...
                              ...                        
9995     le taux de positivité chez le patient symptom...
9996    le villedegrenoble  via le cca  avoir distribu...
9997    considérer comme un  cas contact   roselyne ba...
9998    bonjour  test covid très douloureux ce matin  ...
9999    haha    un main apparaître dans le champ de vi...
Name: Title, Length: 10000, dtype: object

#### Tokenizing and Removing stopwords

In [19]:
from nltk.corpus import stopwords

stopwords = stopwords.words('french')

def rmv_stop_word(tokens):
    return [x for x in tokens if x not in stopwords]

tokens = filtered.map(
    nltk.word_tokenize
).map(
    rmv_stop_word
).map(
    lambda x : " ".join(x)
)

tokens

0                    avoir guérir covid avoir tuer thanos
1       premier symptôme coronavirus être perte goût hein
2       souhaite agréable journer tout monde tout mond...
3                           covid aller bientôt fêter ans
4                   covid avoir passer plus temps fac con
                              ...                        
9995    taux positivité chez patient symptomatique êtr...
9996    villedegrenoble via cca avoir distribuer masqu...
9997    considérer comme cas contact roselyne bachelot...
9998    bonjour test covid très douloureux matin tout ...
9999    haha main apparaître champ vision alexa main n...
Name: Title, Length: 10000, dtype: object

## Keras Pipeline

In [20]:
import tensorflow
from tensorflow.keras import layers

from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

training_data = tokens.to_numpy(dtype = str)

2020-10-21 10:17:31,385 - tensorflow - DEBUG - Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.


In [65]:
training_data

array(['avoir guérir covid avoir tuer thanos',
       'premier symptôme coronavirus être perte goût hein',
       'souhaite agréable journer tout monde tout monde sauf celui avoir céder pression dire covid',
       ...,
       'considérer comme cas contact roselyne bachelot passer sept prochain jour isoler domicile',
       'bonjour test covid très douloureux matin tout cela être faute sidiki bourreau',
       'haha main apparaître champ vision alexa main normal excepter être vert coronavirus avoir prendre corps bob savoir nom être trop compliquer reste vie plus être vénère corona'],
      dtype='<U370')

In [22]:
## Note: can add ngrams if needed
vectorizer = TextVectorization(output_mode = 'int')
## will learn vocab
vectorizer.adapt(training_data)
integer_data = vectorizer(training_data)

print(integer_data[0])

tf.Tensor(
[   4  853    3    4  447 6065    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0], shape=(53,), dtype=int64)


In [78]:
vectorizer.get_vocabulary()

['',
 '[UNK]',
 'être',
 'covid',
 'avoir',
 'plus',
 'trump',
 'faire',
 'tout',
 'aller',
 'coronavirus',
 'pouvoir',
 'positif',
 'cas',
 'nouveau',
 'contre',
 'cela',
 'donald',
 'dire',
 'si',
 'france',
 '️',
 'pourcent',
 'jour',
 'test',
 'devoir',
 'alerte',
 'personne',
 'depuis',
 'masque',
 'comme',
 'santé',
 'sanitaire',
 'paris',
 'après',
 'bien',
 'hôpital',
 'mesure',
 'crise',
 'tester',
 'virus',
 'h',
 '🔴',
 'gouvernement',
 'sans',
 'quand',
 'mettre',
 'an',
 'bon',
 'prendre',
 'très',
 'mort',
 'falloir',
 'octobre',
 'mois',
 'premier',
 'moins',
 'patient',
 'autre',
 'bar',
 'face',
 'voir',
 'décès',
 'président',
 'passer',
 'vouloir',
 'monde',
 'public',
 'non',
 '➡',
 'médecin',
 'annoncer',
 'alors',
 'encore',
 'direct',
 '🇷',
 'semaine',
 'aussi',
 'français',
 '🇫',
 'situation',
 'rien',
 'via',
 'vague',
 'gens',
 'donc',
 'cause',
 'dernier',
 'fermer',
 'vie',
 'celui',
 'savoir',
 'épidémie',
 'marseille',
 'info',
 'selon',
 'pays',
 'école',


In [67]:
vocab_size = len(vectorizer.get_vocabulary())
vocab_size

16829

In [68]:
vocab = vectorizer.get_vocabulary()
reverse_mapping = {}
for i in range(vocab_size):
    reverse_mapping[i] = vocab[i]

In [69]:
sampling_table = tensorflow.keras.preprocessing.sequence.make_sampling_table(vocab_size)
# Should add some 0 padding such that the largest tweet doesn't get skipgramed with the next
(data_tuples, labels) = tensorflow.keras.preprocessing.sequence.skipgrams(
    integer_data.numpy().ravel(),
    vocab_size,
    window_size = 2,
    shuffle = True,
    sampling_table = sampling_table
)

# data_tuples = np.array(data_tuples)
# labels = np.array(labels)

In [70]:
d = [[int(x[0]), int(x[1])] for x in data_tuples]
labels = np.array([int(x) for x in labels])

target_words = np.array(d)[:,0]
context_words = np.array(d)[:,1]

## Model
### Implementation

In [71]:
in_1 = tensorflow.keras.Input(1)
in_2 = tensorflow.keras.Input(1)


E = layers.Embedding(
    vocab_size,
    embed_dim,
    input_length = 1,
    name = 'embeddings'
)

E_1 = E(in_1)
E_2 = E(in_2)


D = tensorflow.keras.backend.sum(E_1 * E_2, axis = -1)

S = layers.Activation(activation = 'sigmoid')(D)

### Validation callback

In [81]:
validation_set = ['président', 'restaurant', 'positif']
validation_set = [vectorizer.get_vocabulary().index(w) for w in validation_set]
validation_set

[63, 106, 12]

In [75]:
class ValidationCallback(tensorflow.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        

TypeError: 'dict' object is not callable

In [73]:
model = tensorflow.keras.Model(
    inputs = [in_1, in_2],
    outputs = [S]
)

model.compile(loss='binary_crossentropy', optimizer='rmsprop')

model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embeddings (Embedding)          (None, 1, 150)       2524350     input_7[0][0]                    
                                                                 input_8[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_Mul_1 (TensorFlowOp [(None, 1, 150)]     0           embeddings[0][0]      

In [74]:
model.fit(
    [target_words, context_words],
    labels,
    batch_size = 64,
    epochs = 10,
    callbacks = [ValidationCallback()],
    shuffle = True
)

Epoch 1/10
4339/4339 [==============================] - 43s 10ms/step - loss: 0.6780
Epoch 2/10
4339/4339 [==============================] - 43s 10ms/step - loss: 0.5987
Epoch 3/10
4339/4339 [==============================] - 43s 10ms/step - loss: 0.5662
Epoch 4/10
4339/4339 [==============================] - 43s 10ms/step - loss: 0.5611
Epoch 5/10
4339/4339 [==============================] - 43s 10ms/step - loss: 0.5571
Epoch 6/10
4339/4339 [==============================] - 43s 10ms/step - loss: 0.5467
Epoch 7/10
4339/4339 [==============================] - 43s 10ms/step - loss: 0.5290
Epoch 8/10
4339/4339 [==============================] - 43s 10ms/step - loss: 0.5052
Epoch 9/10
4339/4339 [==============================] - 43s 10ms/step - loss: 0.4750
Epoch 10/10
4339/4339 [==============================] - 43s 10ms/step - loss: 0.4411


#### similarity layer

In [33]:
sim = K.layers.Dot(axes = (1,1), normalize = True)([target_vec, context_vec])


#### dot product layer

In [34]:
dot_product = K.layers.Dot(axes = (1,1))([target_vec, context_vec])
dot_product = K.layers.Reshape((1,))(dot_product)

#### sigmoid activation function

In [35]:
output = K.layers.Dense(1, activation = 'sigmoid')(dot_product)

### The Model intantiation

In [36]:
model = tf.keras.Model(inputs = [input_target_word, input_context_word], outputs = output)
model.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop')

In [37]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embeddings (Embedding)          (None, 1, 300)       2442000     input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
reshape (Reshape)               (None, 300, 1)       0           embeddings[0][0]      

In [38]:
validation_model = tf.keras.Model(inputs = [input_target_word, input_context_word], outputs = sim)
validation_model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embeddings (Embedding)          (None, 1, 300)       2442000     input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
reshape (Reshape)               (None, 300, 1)       0           embeddings[0][0]      

#### callback

In [39]:
class SimilarityCallback:
    """This callback tracks what words are close to the validation words in the embedded space"""
    def run_sim(self):
        for i in range(valid_size):
            valid_word = int_to_word[valid_examples[i]]
            top_k = 8  # number of nearest neighbors
            sim = self._get_sim(valid_examples[i])
            nearest = (-sim).argsort()[1:top_k + 1]
            log_str = 'Nearest to %s:' % valid_word
            for k in range(top_k):
                close_word = int_to_word[nearest[k]]
                log_str = '%s %s,' % (log_str, close_word)
            print(log_str)

    @staticmethod
    def _get_sim(valid_word_idx):
        sim = np.zeros((vocab_size,))
        in_arr1 = np.zeros((1,))
        in_arr2 = np.zeros((1,))
        for i in range(vocab_size):
            in_arr1[0,] = valid_word_idx
            in_arr2[0,] = i
            out = validation_model.predict_on_batch([in_arr1, in_arr2])
            sim[i] = out
        return sim
sim_cb = SimilarityCallback()

In [41]:
model.predict(np.array([1,2]))

AssertionError: in user code:

    c:\users\artorias_doge\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\keras\engine\training.py:1462 predict_function  *
        return step_function(self, iterator)
    c:\users\artorias_doge\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\keras\engine\training.py:1452 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    c:\users\artorias_doge\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\users\artorias_doge\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\users\artorias_doge\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\distrib

In [129]:
arr_1 = np.zeros((1,))
arr_2 = np.zeros((1,))
arr_3 = np.zeros((1,))
for cnt in range(epochs):
    idx = np.random.randint(0, len(data_labels)-1)
    arr_1[0,] = word_target[idx]
    arr_2[0,] = word_context[idx]
    arr_3[0,] = data_labels[idx]
    loss = model.train_on_batch([arr_1, arr_2], arr_3)
    if i % 100 == 0:
        print("Iteration {}, loss={}".format(cnt, loss))
    if cnt % 10000 == 0:
        sim_cb.run_sim()

TypeError: 'NoneType' object is not callable

#### Code for stemming

In [32]:
## https://stackoverflow.com/questions/13131139/lemmatize-french-text

# from nltk.stem.snowball import FrenchStemmer

# stemmer = FrenchStemmer()

# def stem(tokens):
#     return [stemmer.stem(x) for x in tokens]

# df['Title'] = df['Title'].map(
#     stem
# )